## elections_to_full_data

This code file departs from the election list. We implement several steps in this file. The first objective is to match the names of the elections with the names of the politicians, and also with the names of the non-politicians whose data we have collected. 

The first thing we do is load the packages, and then clean up the dataset and compute the margin of victory of all the politicians. 

In [99]:
# Load the libraries
import pandas as pd
import numpy as np
import re

from pandas_ods_reader import read_ods
import statistics

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import pyplot as plt

from tqdm import tqdm

from functions import *

from thefuzz import fuzz
from thefuzz import process

In [190]:
# Import the election data
elected_people = pd.read_csv("../Data/elections/allelected.csv", encoding='latin-1')
elected_people.head(5)

,achternaam,voornaam,tussenvoegsel,jaar,maand,dag,type verkiezing,districtsnaam,aantal stemmen,omvang_electoraat,zetels,drempel
0,Aalberse,P.J.M.,NaN,1903,2,18,tussentijds,Almelo,3821,7865,1.0,2953.0
1,Aalberse,P.J.M.,NaN,1905,6,16,algemeen,Almelo,5217,9324,1.0,3922.0
2,Aalberse,P.J.M.,NaN,1909,6,11,algemeen,Almelo,4744,9087,1.0,3498.0
3,Aalberse,P.J.M.,NaN,1913,6,17,algemeen,Almelo,5338,10107,1.0,3734.0
4,Akerlaken,D. van,NaN,1850,8,27,algemeen,Hoorn,1319,3430,2.0,1122.0


In [194]:
elected_people['naam'] = elected_people['voornaam'] + ' ' + elected_people['achternaam']
elected_people['verkiezingdatum'] = (elected_people['dag'].astype(str) + 
                                     '-' + 
                                     elected_people['maand'].astype(str) +
                                     '-' +
                                     elected_people['jaar'].astype(str)
                                    )
elected_people['verkiezingdatum'] = elected_people.apply(lambda x: pd.Timestamp(year=x['jaar'], month=x['maand'], day=x['dag']), axis=1)

In [102]:
election_results_details = pd.read_csv("../Data/elections/election_results_details.csv").iloc[:,1:]
election_results_details['Verkiezingdatum'] = (election_results_details['Verkiezingdatum'].
                                               apply(lambda x: pd.Timestamp(x))
                                              )

def get_zetels(df):
    a = pd.to_numeric(df['Aantal zetels'], errors='coerce')
    b = np.mean(a)
    return (b)

aantal_zetels = (election_results_details.groupby(['District', 'Verkiezingdatum']).
                 apply(get_zetels).reset_index().rename(columns={0:'Aantal zetels'})
                )

In [250]:
#pd.merge(elected_people, aantal_zetels, 
#         left_on=['districtsnaam', 'verkiezingdatum'],
#         right_on=['District', 'Verkiezingdatum']).drop(columns=['District', 'Verkiezingdatum'])


In [251]:
all_candidates = pd.read_csv("../Data/elections/election_results_details.csv").iloc[:,1:]


all_candidates['Verkiezingdatum'] = all_candidates['Verkiezingdatum'].str.split("/")
all_candidates['Verkiezingdatum'] = all_candidates['Verkiezingdatum'].apply(lambda x: [int(y) for y in x])
all_candidates['Verkiezingdatum'] = all_candidates['Verkiezingdatum'].apply(
    lambda x: pd.Timestamp(day=x[0], month=x[1], year=x[2]) if all(type(y) == int for y in x)
    else None
)
all_candidates['Aantal zetels'] = all_candidates['Aantal zetels'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
all_candidates['Aantal stemmen'] = (all_candidates['Aantal stemmen'].
                                    apply(lambda x: pd.to_numeric(x, errors='coerce'))
                                   )

aantal_stemmen = (all_candidates.groupby(['District','Verkiezingdatum']).
 apply(lambda x: sum(x['Aantal stemmen'])).
 reset_index().
 rename(columns={0:'totaal aantal stemmen'})
)

all_candidates = pd.merge(all_candidates, aantal_stemmen,
        left_on=['District', 'Verkiezingdatum'],
        right_on=['District', 'Verkiezingdatum'])

all_candidates = (all_candidates.groupby(['District', 'Verkiezingdatum']).
 apply(lambda x: x.sort_values(['Aantal stemmen'], 
                               ascending = False))
).reset_index(drop=True)

all_candidates['hoeveelste_in_verkiezing'] = (all_candidates.groupby(['District', 'Verkiezingdatum']).
                                              cumcount() + 1)

all_candidates['gewonnen'] = np.where(all_candidates['hoeveelste_in_verkiezing'] <= all_candidates['Aantal zetels'], 1, 0)
all_candidates['marginal_winner'] = np.where(all_candidates['Aantal zetels'] - all_candidates['hoeveelste_in_verkiezing'] == 0, 1, 0)
all_candidates['marginal_loser'] = np.where(all_candidates['Aantal zetels'] - all_candidates['hoeveelste_in_verkiezing'] == -1, 1, 0)

In [252]:
all_candidates = get_margin(all_candidates)

100%|██████████| 8238/8238 [01:21<00:00, 101.21it/s]


In [168]:
#get_match(elected_people).to_csv("../Data/politician_data/key_allelected_to_all_candidates.csv", index = False)

100%|██████████| 639/639 [00:04<00:00, 134.42it/s]


In [253]:
key = pd.read_csv("../Data/politician_data/key_allelected_to_all_candidates.csv")

pd.merge(all_candidates, key, 
         how='left',
         left_on='Naam',
         right_on='name_in_all_elections').to_csv("test.csv")

In [8]:

def get_elec_stats(dataframe):
    
    out = pd.DataFrame()
    
    for i in tqdm(range(len(dataframe))):
        
        naam = dataframe.iloc[i]['Naam']
        datum = dataframe.iloc[i]['Verkiezingdatum']
        
        # proberen
        hoeveelste_keer_prob = dataframe[(dataframe['Naam'] == naam) & (dataframe['Verkiezingdatum'] <= datum)].shape[0]
        hoeveelste_keer_prob_alg_per = dataframe[(dataframe['Naam'] == naam) & (dataframe['Verkiezingdatum'] <= datum) & ( (dataframe['Type'] == 'algemeen') | (dataframe['Type'] == 'periodiek'))].shape[0]
          
        hoeveel_keer_prob_tot =
        hoelang_tussen_1_en_2 = 
        hoelang_tussen_1_en_laatst = 
        
        # gewonnen
        
        if dataframe.iloc[i]['gewonnen'] == 1:
            
            
            hoeveelste_keer_gewonnen = dataframe[(dataframe['Naam'] == naam) & 
                                             (dataframe['gewonnen'] == 1) & 
                                             (dataframe['Verkiezingdatum'] <= datum)].shape[0]
            hoeveelste_keer_gewonnen_alg = dataframe[(dataframe['Naam'] == naam) & 
                                                 (dataframe['gewonnen'] == 1) & 
                                                 (dataframe['Verkiezingdatum'] <= datum) & 
                                                 ((dataframe['Type'] == 'algemeen') | 
                                                  (dataframe['Type'] == 'periodiek'))].shape[0]
        else:
            
            hoeveelste_keer_gewonnen = None
            hoeveelste_keer_gewonnen_alg = None
        
        hoevaak_gewonnen_alltime = dataframe[(dataframe['Naam'] == naam) & 
                                            (dataframe['gewonnen'] == 1) & 
                                            (dataframe['Type'] == 'algemeen')].shape[0]
            
        hoevaak_gewonnen_toekomst = dataframe[(dataframe['Naam'] == naam) & 
                                             (dataframe['gewonnen'] == 1) & 
                                             (dataframe['Verkiezingdatum'] > datum)].shape[0]
        hoevaak_gewonnen_verleden = dataframe[(dataframe['Naam'] == naam) & 
                                             (dataframe['gewonnen'] == 1) & 
                                             (dataframe['Verkiezingdatum'] < datum)].shape[0]
            
        # volgende verkiezingen 
        elections_participated_in_future = dataframe[(dataframe['Naam'] == naam) & 
                                                           (dataframe['Verkiezingdatum'] > datum) &
                                                           ((dataframe['Type'] == 'algemeen') | (dataframe['Type'] == 'periodiek'))]['Verkiezingdatum'].tolist()
        verk_2_6_gewonnen = [None, None, None, None, None]
        for i, val in enumerate(elections_participated_in_future):
            
                cur_gewonnen = all_candidates[(all_candidates['Naam'] == naam) & (all_candidates['Verkiezingdatum'] == val)]['gewonnen'].values[0]
                verk_2_6_gewonnen[i] = cur_gewonnen               
        
        # put everything together in dataframe
        interim = dataframe.iloc[i:i+1]
        
        interim = interim.assign(hoeveelste_keer_prob = hoeveelste_keer_prob)
        interim = interim.assign(hoeveelste_keer_prob_alg = hoeveelste_keer_prob_alg_per)
        
        interim = interim.assign(hoeveelste_keer_gewonnen = hoeveelste_keer_gewonnen)
        interim = interim.assign(hoeveelste_keer_gewonnen_alg = hoeveelste_keer_gewonnen_alg)
        interim = interim.assign(hoevaak_gewonnen_alltime = hoevaak_gewonnen_alltime)
        interim = interim.assign(hoevaak_gewonnen_verleden = hoevaak_gewonnen_verleden)
        interim = interim.assign(hoevaak_gewonnen_toekomst = hoevaak_gewonnen_toekomst)
        
                                                         
        out = out.append(interim)
        

,Naam,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,Opkomst,Aantal stembriefjes,Aantal stemmen geldig,Aantal stemmen blanco,Aantal zetels,Kiesdrempel,totaal aantal stemmen,hoeveelste_in_verkiezing,gewonnen,marginal_winner,marginal_loser,margin
0,S.A. de Moraaz,NaN,503.0,52.84%,Alkmaar,1848-11-30,algemeen,1107,956,956,952,4,1.0,476,941.0,1,1,1,0,0.069075
1,G. van Leeuwen,NaN,438.0,46.01%,Alkmaar,1848-11-30,algemeen,1107,956,956,952,4,1.0,476,941.0,2,0,0,1,-0.069075
2,mr. H.J. Smit,NaN,1566.0,79.86%,Alkmaar,1850-08-27,algemeen,2833,1977,1979,3922,20,2.0,981,3781.0,1,1,0,0,0.233007
3,S.A. de Moraaz,NaN,1275.0,65.02%,Alkmaar,1850-08-27,algemeen,2833,1977,1979,3922,20,2.0,981,3781.0,2,1,1,0,0.156043
4,jhr.mr. C. van Foreest,NaN,685.0,34.93%,Alkmaar,1850-08-27,algemeen,2833,1977,1979,3922,20,2.0,981,3781.0,3,0,0,1,-0.156043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8233,B. Luteraan,SDP,32.0,0.41%,Zwolle,1913-06-17,algemeen,8889,-,7897,7786,111,1.0,3893,7786.0,4,0,0,0,-0.494606
8234,F.M. Knobel,VL(Lib/VD/SDAP),4249.0,50.15%,Zwolle,1913-06-25,herstemming,8889,-,8516,8472,44,1.0,4236,8472.0,1,1,1,0,0.003069
8235,A. baron van Dedem,CHU(Ka/AR),4223.0,49.85%,Zwolle,1913-06-25,herstemming,8889,-,8516,8472,44,1.0,4236,8472.0,2,0,0,1,-0.003069
8236,F.M. Knobel,VL,3236.0,86.22%,Zwolle,1917-06-15,algemeen,9645,-,3967,3753,214,1.0,1877,3753.0,1,1,1,0,0.724487


In [84]:

    
    
# merge the elected_people with new dataframe

# from all_candidates, find out who has ever become a politician

# find out which elections are consequential, which ones aren't


# from the ones which aren't, still use the filter
